In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
insurance_data = pd.read_csv('Insurance.csv')

In [ ]:
insurance_data.head()

In [ ]:
insurance_data.isnull().sum()

In [ ]:
insurance_data.info()

In [ ]:
insurance_data.describe()

In [ ]:
insurance_data.isnull().sum()

In [ ]:
sns.boxplot(x='smoker', y='charges', data=insurance_data)
plt.title('Insurance Charges by Smoking Status')
plt.show()

In [ ]:
numeric_cols = insurance_data.select_dtypes(include=['number'])
plt.figure(figsize=(10, 8))
sns.heatmap(numeric_cols.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap of Numerical Features")
plt.tight_layout()
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.displot(insurance_data['age'])
plt.show()

In [ ]:
insurance_data['age'].value_counts()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.displot(insurance_data['bmi'])
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.displot(insurance_data['children'])
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.countplot(x= insurance_data['sex'], data = insurance_data)
plt.show()

In [ ]:
insurance_data['sex'].value_counts()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.countplot(x= insurance_data['smoker'], data = insurance_data)
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(6,6))
sns.countplot(x= insurance_data['region'], data = insurance_data)
plt.show()

In [ ]:
insurance_data['region'].value_counts()

In [ ]:
insurance_data

In [ ]:
insurance_data.columns


In [ ]:
# One Hot Encoding
dataset = pd.get_dummies(data=insurance_data, drop_first=True)

In [ ]:
x= dataset.drop(columns='charges')
y =dataset['charges']

In [ ]:
x

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=0)

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x

In [ ]:
from sklearn.metrics import r2_score
rf = RandomForestRegressor(random_state=0)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
r2_tuning = r2_score(y_test, y_pred)
print("R2 Score (Random Forest):", r2_tuning)


In [ ]:
from sklearn.metrics import r2_score
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=0),
                           param_grid=param_grid,
                           cv=5,
                           scoring='r2',
                           n_jobs=-1,
                           verbose=1)

grid_search.fit(x_train, y_train)
best_rf_model = grid_search.best_estimator_

y_pred_rf = best_rf_model.predict(x_test)
print("R2 Score (Random Forest - Tuned):", r2_score(y_test, y_pred_rf))


In [ ]:
from sklearn.linear_model import LinearRegression
regressor_lr = LinearRegression()
regressor_lr.fit(x_train, y_train)

In [ ]:
y_pred = regressor_lr.predict(x_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
from xgboost import XGBRFRegressor
regressor_xgb = XGBRFRegressor()
regressor_xgb.fit(x_train, y_train)

In [ ]:
y_pred = regressor_xgb.predict(x_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

models = {
    "Random Forest": best_rf_model,
    "Linear Regression": regressor_lr,
    "XGBoost Regressor": regressor_xgb
}

results = []

for name, model in models.items():
    y_pred = model.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    
    results.append({
        "Model": name,
        "R² Score": round(r2, 4),
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2)
    })
comparison_df = pd.DataFrame(results)
print(comparison_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
error_metrics_df = comparison_df.melt(id_vars='Model', value_vars=['MAE', 'RMSE'],
                                      var_name='Metric', value_name='Value')

plt.figure(figsize=(8, 5))
sns.barplot(data=error_metrics_df, x='Model', y='Value', hue='Metric')
plt.title('Model Comparison: MAE vs RMSE')
plt.ylabel('Error Value')
plt.xlabel('Model')
plt.legend(title='Metric')
plt.tight_layout()
plt.show()




In [ ]:
model_colors = {
    "Random Forest": "tomato",
    "Linear Regression": "skyblue",
    "XGBoost Regressor": "mediumseagreen"
}

plt.figure(figsize=(6, 5))
sns.barplot(
    data=comparison_df,
    x='Model',
    y='R² Score (%)',
    palette=model_colors 
)

plt.title('Model Comparison: R² Score (%)')
plt.ylabel('R² (%)')
plt.xlabel('Model')
plt.ylim(0, 100)
plt.tight_layout()
plt.show()


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [ ]:
dataset.head()

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
xgb_model = XGBRegressor(random_state=0)

grid_search = GridSearchCV(estimator=xgb_model,
                           param_grid=param_grid,
                           scoring='r2',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(x_train, y_train)
best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(x_test)

print("Best Parameters:", grid_search.best_params_)
print(" R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))


In [ ]:
regressor_xgb.predict(sc.transform(frank_obs))

In [ ]:
sara_obs = [[19, 27, 0, 0, 0, 1, 0, 0]]

In [ ]:
regressor_xgb.predict(sc.transform(sara_obs))

In [ ]:
import joblib
joblib.dump(best_xgb_model, 'MIPML.pkl')

